# Langchain、AnalyticDB、OpenAIを使った質問応答システム
このノートブックでは、Langchain、知識ベースとしてのAnalyticDB、OpenAI埋め込みを使用して質問応答システムを実装する方法を紹介します。AnalyticDBに馴染みがない場合は、[Getting_started_with_AnalyticDB_and_OpenAI.ipynb](Getting_started_with_AnalyticDB_and_OpenAI.ipynb)ノートブックを確認することをお勧めします。

このノートブックでは、以下のエンドツーエンドのプロセスを紹介します：
- OpenAI APIを使用した埋め込みの計算
- 知識ベースを構築するためのAnalyticDBインスタンスへの埋め込みの保存
- OpenAI APIを使用した生テキストクエリの埋め込みへの変換
- 作成されたコレクションで最近傍検索を実行してコンテキストを見つけるためのAnalyticDBの使用
- 与えられたコンテキストで答えを見つけるためのLLMへの質問

すべてのステップは、対応するLangchainメソッドの呼び出しに簡略化されます。

## 前提条件
この演習を行うために、いくつかの準備が必要です：
[AnalyticDB クラウドインスタンス](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/latest/product-introduction-overview)。
フレームワークとしての [Langchain](https://github.com/hwchase17/langchain)。
OpenAI API キー。

### 必要なパッケージのインストール
このノートブックには以下のPythonパッケージが必要です：`openai`、`tiktoken`、`langchain`、`psycopg2cffi`。
- `openai`はOpenAI APIへの便利なアクセスを提供します。
- `tiktoken`はOpenAIのモデルで使用するための高速なBPEトークナイザーです。
- `langchain`はLLMを使ったアプリケーションをより簡単に構築するのに役立ちます。
- `psycopg2cffi`ライブラリはベクターデータベースとの相互作用に使用されますが、他のPostgreSQLクライアントライブラリでも問題ありません。

In [ ]:
! pip install openai tiktoken langchain psycopg2cffi 

In [2]:
! export OPENAI_API_KEY="your API key"

In [1]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


### OpenAI APIキーの準備
OpenAI APIキーは、ドキュメントとクエリのベクトル化に使用されます。

OpenAI APIキーをお持ちでない場合は、[https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys) から取得できます。

キーを取得したら、以下のコマンドを実行して環境変数として `OPENAI_API_KEY` に追加してください：

### AnalyticDB接続文字列の準備
AnalyticDB接続文字列を構築するには、以下のパラメータが必要です：`PG_HOST`、`PG_PORT`、`PG_DATABASE`、`PG_USER`、および`PG_PASSWORD`。正しい接続文字列を設定するために、まずこれらをエクスポートする必要があります。その後、接続文字列を構築します。

In [4]:
! export PG_HOST="your AnalyticDB host url"
! export PG_PORT=5432 # Optional, default value is 5432
! export PG_DATABASE=postgres # Optional, default value is postgres
! export PG_USER="your username"
! export PG_PASSWORD="your password"

In [2]:
import os
from langchain.vectorstores.analyticdb import AnalyticDB

CONNECTION_STRING = AnalyticDB.connection_string_from_db_params(
    driver=os.environ.get("PG_DRIVER", "psycopg2cffi"),
    host=os.environ.get("PG_HOST", "localhost"),
    port=int(os.environ.get("PG_PORT", "5432")),
    database=os.environ.get("PG_DATABASE", "postgres"),
    user=os.environ.get("PG_USER", "postgres"),
    password=os.environ.get("PG_PASSWORD", "postgres"),
)

In [3]:
import json

with open("questions.json", "r") as fp:
    questions = json.load(fp)

with open("answers.json", "r") as fp:
    answers = json.load(fp)

## データの読み込み
このセクションでは、自然な質問とその回答を含むデータを読み込みます。すべてのデータは、AnalyticDBを知識ベースとしたLangchainアプリケーションの作成に使用されます。

In [4]:
print(questions[0])

when is the last episode of season 8 of the walking dead


In [ ]:
import wget

# All the examples come from https://ai.google.com/research/NaturalQuestions
# This is a sample of the training set that we download and extract for some
# further processing.
wget.download("https://storage.googleapis.com/dataset-natural-questions/questions.json")
wget.download("https://storage.googleapis.com/dataset-natural-questions/answers.json")

In [6]:
print(answers[0])

No . overall No. in season Title Directed by Written by Original air date U.S. viewers ( millions ) 100 `` Mercy '' Greg Nicotero Scott M. Gimple October 22 , 2017 ( 2017 - 10 - 22 ) 11.44 Rick , Maggie , and Ezekiel rally their communities together to take down Negan . Gregory attempts to have the Hilltop residents side with Negan , but they all firmly stand behind Maggie . The group attacks the Sanctuary , taking down its fences and flooding the compound with walkers . With the Sanctuary defaced , everyone leaves except Gabriel , who reluctantly stays to save Gregory , but is left behind when Gregory abandons him . Surrounded by walkers , Gabriel hides in a trailer , where he is trapped inside with Negan . 101 `` The Damned '' Rosemary Rodriguez Matthew Negrete & Channing Powell October 29 , 2017 ( 2017 - 10 - 29 ) 8.92 Rick 's forces split into separate parties to attack several of the Saviors ' outposts , during which many members of the group are killed ; Eric is critically injure

## チェーン定義

LangchainはすでにAnalyticDBと統合されており、与えられたドキュメントのリストに対してすべてのインデックス作成を実行します。今回のケースでは、保有している回答のセットを保存する予定です。

In [7]:
from langchain.vectorstores import AnalyticDB
from langchain.embeddings import OpenAIEmbeddings
from langchain import VectorDBQA, OpenAI

embeddings = OpenAIEmbeddings()
doc_store = AnalyticDB.from_texts(
    texts=answers, embedding=embeddings, connection_string=CONNECTION_STRING,
    pre_delete_collection=True,
)

この段階では、すべての可能な回答がすでにAnalyticDBに保存されているため、QAチェーン全体を定義することができます。

In [9]:
from langchain.chains import RetrievalQA

llm = OpenAI()
qa = VectorDBQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=doc_store,
    return_source_documents=False,
)

## データの検索

データがAnalyticDBに格納されると、質問を開始できます。質問は自動的にOpenAIモデルによってベクトル化され、作成されたベクトルはAnalyticDB内で一致する可能性のある回答を見つけるために使用されます。取得されると、最も類似した回答がOpenAI Large Language Modelに送信されるプロンプトに組み込まれます。

In [10]:
import random

random.seed(52)
selected_questions = random.choices(questions, k=5)

In [11]:
for question in selected_questions:
    print(">", question)
    print(qa.run(question), end="\n\n")

> where do frankenstein and the monster first meet
 Victor retreats into the mountains, and that is where the Creature finds him and pleads for Victor to hear his tale.

> who are the actors in fast and furious
 The main cast of Fast & Furious includes Vin Diesel as Dominic Toretto, Paul Walker as Brian O'Conner, Michelle Rodriguez as Letty Ortiz, Jordana Brewster as Mia Toretto, Tyrese Gibson as Roman Pearce, and Ludacris as Tej Parker.

> properties of red black tree in data structure
 The properties of a red-black tree in data structure are that each node is either red or black, the root is black, all leaves (NIL) are black, and if a node is red, then both its children are black. Additionally, every path from a given node to any of its descendant NIL nodes contains the same number of black nodes.

> who designed the national coat of arms of south africa
 Iaan Bekker

> caravaggio's death of the virgin pamela askew
 I don't know.



### カスタムプロンプトテンプレート

Langchainの`stuff`チェーンタイプは、質問とコンテキストドキュメントが組み込まれた特定のプロンプトを使用します。デフォルトのプロンプトは以下のようになっています：

```text
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:
```

しかし、独自のプロンプトテンプレートを提供し、`stuff`チェーンタイプを使用しながらOpenAI LLMの動作を変更することができます。`{context}`と`{question}`をプレースホルダーとして保持することが重要です。

#### カスタムプロンプトの実験

異なるプロンプトテンプレートを使用して、モデルが以下のように動作するようにできます：
1. 答えを知っている場合は、一文で回答する。
2. 質問の答えがわからない場合は、ランダムな楽曲タイトルを提案する。

In [12]:
from langchain.prompts import PromptTemplate
custom_prompt = """
Use the following pieces of context to answer the question at the end. Please provide
a short single-sentence summary answer only. If you don't know the answer or if it's
not present in given context, don't try to make up an answer, but suggest me a random
unrelated song title I could listen to.
Context: {context}
Question: {question}
Helpful Answer:
"""

custom_prompt_template = PromptTemplate(
    template=custom_prompt, input_variables=["context", "question"]
)

In [13]:
custom_qa = VectorDBQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=doc_store,
    return_source_documents=False,
    chain_type_kwargs={"prompt": custom_prompt_template},
)

In [14]:
random.seed(41)
for question in random.choices(questions, k=5):
    print(">", question)
    print(custom_qa.run(question), end="\n\n")

> what was uncle jesse's original last name on full house
Uncle Jesse's original last name on Full House was Cochran.

> when did the volcano erupt in indonesia 2018
No information about a volcano erupting in Indonesia in 2018 is present in the given context. Suggested song title: "Volcano" by U2.

> what does a dualist way of thinking mean
A dualist way of thinking means believing that humans possess a non-physical mind or soul which is distinct from their physical body.

> the first civil service commission in india was set up on the basis of recommendation of
The first Civil Service Commission in India was not set up on the basis of a recommendation.

> how old do you have to be to get a tattoo in utah
In Utah, you must be at least 18 years old to get a tattoo.

